In [1]:
%load_ext autoreload

In [11]:
%autoreload 2
from tpot import TPOT

tpot = TPOT(generations=100)

ind = tpot.toolbox.individual()
g = ind[0]

In [12]:
ind

 ...]

In [13]:
g.arity

3

In [14]:
ind[1].name

'dt_feature_selection'

In [15]:
str(ind)

'random_forest(dt_feature_selection(dt_feature_selection(combine_dfs(random_forest(combine_dfs(combine_dfs(combine_dfs(dt_feature_selection(ARG0, 79), dt_feature_selection(ARG0, 48)), dt_feature_selection(decision_tree(ARG0, 69, 62), add(86, 77))), dt_feature_selection(combine_dfs(decision_tree(ARG0, 42, 79), random_forest(ARG0, 59, 83)), mul(add(53, 43), sub(72, 54)))), add(sub(sub(sub(38, 54), sub(65, 47)), add(mul(8, 68), add(45, 42))), sub(mul(sub(41, 30), mul(14, 100)), sub(mul(50, 37), sub(78, 56)))), mul(add(mul(mul(99, 70), sub(11, 70)), add(sub(46, 38), mul(75, 22))), add(sub(add(14, 62), sub(10, 75)), mul(mul(23, 40), mul(49, 14))))), random_forest(decision_tree(combine_dfs(random_forest(combine_dfs(ARG0, ARG0), add(87, 51), sub(44, 1)), random_forest(decision_tree(ARG0, 70, 6), mul(7, 98), mul(77, 46))), mul(mul(mul(82, 22), sub(46, 70)), add(add(81, 83), sub(86, 18))), sub(mul(add(4, 52), mul(9, 72)), sub(sub(66, 50), sub(48, 81)))), add(mul(sub(sub(79, 76), mul(77, 17)), m

In [20]:
import deap

def unroll_pipeline(new_pipeline):
    # replace all of the mathematical operators with their results
    while True:
        for i in range(len(new_pipeline) - 1, -1, -1):
            node = new_pipeline[i]
            if type(node) == deap.gp.Primitive and node.name in ['add', 'sub', 'mul']:
                val1 = int(new_pipeline[i + 1].name)
                val2 = int(new_pipeline[i + 2].name)
                if node.name == 'add':
                    new_val = val1 + val2
                elif node.name == 'sub':
                    new_val = val1 - val2
                else:
                    new_val = val1 * val2

                new_val = deap.gp.Terminal(symbolic=new_val, terminal=new_val, ret=new_val)
                new_pipeline = new_pipeline[:i] + [new_val] + new_pipeline[i + 3:]
                break
        else:
            break

    # unroll the nested function calls into serial code
    pipeline_list = []
    result_num = 1
    while True:
        for node_index in range(len(new_pipeline) - 1, -1, -1):
            node = new_pipeline[node_index]
            if type(node) != deap.gp.Primitive:
                continue

            node_params = new_pipeline[node_index + 1:node_index + node.arity + 1]

            new_val = 'result{}'.format(result_num)
            operator_list = [new_val, node.name]
            operator_list.extend([x.name for x in node_params])
            pipeline_list.append(operator_list)
            result_num += 1
            new_val = deap.gp.Terminal(symbolic=new_val, terminal=new_val, ret=new_val)
            new_pipeline = new_pipeline[:node_index] + [new_val] + new_pipeline[node_index + node.arity + 1:]
            break
        else:
            break
    
    # replace the function calls with the corresponding code
    pipeline_text = '''from itertools import combinations

import numpy as np
import pandas as pd

from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# NOTE: Make sure that the class is labeled 'class' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR')
training_indeces, testing_indeces = next(iter(StratifiedShuffleSplit(tpot_data['class'].values, n_iter=1, train_size=0.75)))

'''
    for operator in pipeline_list:
        operator_num = int(operator[0].strip('result'))
        result_name = operator[0]
        operator_name = operator[1]
        operator_text = ''
        
        # make copies of the data set for each reference to ARG0
        if operator[2] == 'ARG0':
            operator[2] = 'result{}'.format(operator_num)
            operator_text += '{} = tpot_data.copy()\n'.format(operator[2])

        if operator[3] == 'ARG0':
            operator[3] = 'result{}'.format(operator_num)
            operator_text += '{} = tpot_data.copy()\n'.format(operator[3])


        # replace the TPOT functions with their corresponding code
        if operator_name == 'decision_tree':
            max_features = int(operator[3])
            max_depth = int(operator[4])
            
            if max_features < 1:
                max_features = '\'auto\''
            elif max_features == 1:
                max_features = None

            if max_depth < 1:
                max_depth = None
            
            operator_text += '\ndtc{} = DecisionTreeClassifier(max_features=min({}, len({}.columns) - 1), max_depth={})\n'.format(operator_num, max_features, operator[2], max_depth)
            operator_text += '''dtc{0}.fit({1}.loc[training_indeces].drop('class', axis=1).values, {1}.loc[training_indeces]['class'].values)\n'''.format(operator_num, operator[2])
            if result_name != operator[2]:
                operator_text += '{} = {}\n'.format(result_name, operator[2])
            operator_text += '''{0}['dtc{1}-classification'] = dtc{1}.predict({0}.drop('class', axis=1).values)\n'''.format(result_name, operator_num)
        
        elif operator_name == 'random_forest':
            num_trees = int(operator[3])
            max_features = int(operator[4])
            
            if num_trees < 1:
                num_trees = 1
            elif num_trees > 500:
                num_trees = 500

            if max_features < 1:
                max_features = '\'auto\''
            elif max_features == 1:
                max_features = None
            
            operator_text += '\nrfc{} = RandomForestClassifier(n_estimators={}, max_features=min({}, len({}.columns) - 1))\n'.format(operator_num, num_trees, max_features, operator[2])
            operator_text += '''rfc{0}.fit({1}.loc[training_indeces].drop('class', axis=1).values, {1}.loc[training_indeces]['class'].values)\n'''.format(operator_num, operator[2])
            if result_name != operator[2]:
                operator_text += '{} = {}\n'.format(result_name, operator[2])
            operator_text += '''{0}['rfc{1}-classification'] = rfc{1}.predict({0}.drop('class', axis=1).values)\n'''.format(result_name, operator_num)
        
        elif operator_name == 'combine_dfs':
            operator_text += '{2} = {0}.join({1}[[column for column in {1}.columns.values if column not in {0}.columns.values]])\n'.format(operator[2], operator[3], result_name)
        
        elif operator_name == 'subset_df':
            start = int(operator[3])
            stop = int(operator[4])
            if stop <= start:
                stop = start + 1
            operator_text += '\nsubset_df1 = {0}[sorted({0}.columns.values)[{1}:{2}]]\n'.format(operator[2], start, stop)
            operator_text += '''subset_df2 = {0}[[column for column in ['class'] if column not in subset_df1.columns.values]]\n'''.format(operator[2])
            operator_text += '{} = subset_df1.join(subset_df2)\n'.format(result_name)
        
        elif operator_name == 'dt_feature_selection':
            operator_text += '''
training_features = {0}.loc[training_indeces].drop('class', axis=1)
training_class_vals = {0}.loc[training_indeces, 'class'].values

pair_scores = dict()
for features in combinations({0}.columns.values, 2):
    dtc = DecisionTreeClassifier()
    training_feature_vals = training_features[list(features)].values
    dtc.fit(training_feature_vals, training_class_vals)
    pair_scores[features] = (dtc.score(training_feature_vals, training_class_vals), list(features))

best_pairs = []
for pair in sorted(pair_scores, key=pair_scores.get, reverse=True)[:{1}]:
    best_pairs.extend(list(pair))
best_pairs = sorted(list(set(best_pairs)))

{2} = {0}[sorted(list(set(best_pairs + ['class'])))]
'''.format(operator[2], operator[3], result_name)
        
        pipeline_text += operator_text
    
    return pipeline_text

print(unroll_pipeline(ind))

from itertools import combinations

import numpy as np
import pandas as pd

from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# NOTE: Make sure that the class is labeled 'class' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR')
training_indeces, testing_indeces = next(iter(StratifiedShuffleSplit(tpot_data['class'].values, n_iter=1, train_size=0.75)))

result1 = tpot_data.copy()

dtc1 = DecisionTreeClassifier(max_features=min(70, len(result1.columns) - 1), max_depth=6)
dtc1.fit(result1.loc[training_indeces].drop('class', axis=1).values, result1.loc[training_indeces]['class'].values)
result1['dtc1-classification'] = dtc1.predict(result1.drop('class', axis=1).values)

rfc2 = RandomForestClassifier(n_estimators=500, max_features=min(3542, len(result1.columns) - 1))
rfc2.fit(result1.loc[training_indeces].drop('class', axis=1).values, result1.loc[

In [21]:
from itertools import combinations

import numpy as np
import pandas as pd

from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# NOTE: Make sure that the class is labeled 'class' in the data file
tpot_data = pd.read_csv('/Users/randal_olson/Dropbox/Private/research/tpot/GAMETES/4x2-way_her-0.1_pop-1600_attribs-100_discrete_EDM-1/4x2-way_her-0.1_pop-1600_attribs-100_discrete_EDM-1_01.txt', sep='\t')
tpot_data.rename(columns={'Class': 'class'}, inplace=True)
training_indeces, testing_indeces = next(iter(StratifiedShuffleSplit(tpot_data['class'].values, n_iter=1, train_size=0.75)))

result1 = tpot_data.copy()

dtc1 = DecisionTreeClassifier(max_features=min(70, len(result1.columns) - 1), max_depth=6)
dtc1.fit(result1.loc[training_indeces].drop('class', axis=1).values, result1.loc[training_indeces]['class'].values)
result1['dtc1-classification'] = dtc1.predict(result1.drop('class', axis=1).values)

rfc2 = RandomForestClassifier(n_estimators=500, max_features=min(3542, len(result1.columns) - 1))
rfc2.fit(result1.loc[training_indeces].drop('class', axis=1).values, result1.loc[training_indeces]['class'].values)
result2 = result1
result2['rfc2-classification'] = rfc2.predict(result2.drop('class', axis=1).values)
result3 = tpot_data.copy()
result3 = tpot_data.copy()
result3 = result3.join(result3[[column for column in result3.columns.values if column not in result3.columns.values]])

rfc4 = RandomForestClassifier(n_estimators=138, max_features=min(43, len(result3.columns) - 1))
rfc4.fit(result3.loc[training_indeces].drop('class', axis=1).values, result3.loc[training_indeces]['class'].values)
result4 = result3
result4['rfc4-classification'] = rfc4.predict(result4.drop('class', axis=1).values)
result5 = result4.join(result2[[column for column in result2.columns.values if column not in result4.columns.values]])

dtc6 = DecisionTreeClassifier(max_features=min('auto', len(result5.columns) - 1), max_depth=36239)
dtc6.fit(result5.loc[training_indeces].drop('class', axis=1).values, result5.loc[training_indeces]['class'].values)
result6 = result5
result6['dtc6-classification'] = dtc6.predict(result6.drop('class', axis=1).values)

rfc7 = RandomForestClassifier(n_estimators=500, max_features=min('auto', len(result6.columns) - 1))
rfc7.fit(result6.loc[training_indeces].drop('class', axis=1).values, result6.loc[training_indeces]['class'].values)
result7 = result6
result7['rfc7-classification'] = rfc7.predict(result7.drop('class', axis=1).values)
result8 = tpot_data.copy()

rfc8 = RandomForestClassifier(n_estimators=59, max_features=min(83, len(result8.columns) - 1))
rfc8.fit(result8.loc[training_indeces].drop('class', axis=1).values, result8.loc[training_indeces]['class'].values)
result8['rfc8-classification'] = rfc8.predict(result8.drop('class', axis=1).values)
result9 = tpot_data.copy()

dtc9 = DecisionTreeClassifier(max_features=min(42, len(result9.columns) - 1), max_depth=79)
dtc9.fit(result9.loc[training_indeces].drop('class', axis=1).values, result9.loc[training_indeces]['class'].values)
result9['dtc9-classification'] = dtc9.predict(result9.drop('class', axis=1).values)
result10 = result9.join(result8[[column for column in result8.columns.values if column not in result9.columns.values]])

training_features = result10.loc[training_indeces].drop('class', axis=1)
training_class_vals = result10.loc[training_indeces, 'class'].values

pair_scores = dict()
for features in combinations(result10.columns.values, 2):
    dtc = DecisionTreeClassifier()
    training_feature_vals = training_features[list(features)].values
    dtc.fit(training_feature_vals, training_class_vals)
    pair_scores[features] = (dtc.score(training_feature_vals, training_class_vals), list(features))

best_pairs = []
for pair in sorted(pair_scores, key=pair_scores.get, reverse=True)[:1728]:
    best_pairs.extend(list(pair))
best_pairs = sorted(list(set(best_pairs)))

result11 = result10[sorted(list(set(best_pairs + ['class'])))]
result12 = tpot_data.copy()

dtc12 = DecisionTreeClassifier(max_features=min(69, len(result12.columns) - 1), max_depth=62)
dtc12.fit(result12.loc[training_indeces].drop('class', axis=1).values, result12.loc[training_indeces]['class'].values)
result12['dtc12-classification'] = dtc12.predict(result12.drop('class', axis=1).values)

training_features = result12.loc[training_indeces].drop('class', axis=1)
training_class_vals = result12.loc[training_indeces, 'class'].values

pair_scores = dict()
for features in combinations(result12.columns.values, 2):
    dtc = DecisionTreeClassifier()
    training_feature_vals = training_features[list(features)].values
    dtc.fit(training_feature_vals, training_class_vals)
    pair_scores[features] = (dtc.score(training_feature_vals, training_class_vals), list(features))

best_pairs = []
for pair in sorted(pair_scores, key=pair_scores.get, reverse=True)[:163]:
    best_pairs.extend(list(pair))
best_pairs = sorted(list(set(best_pairs)))

result13 = result12[sorted(list(set(best_pairs + ['class'])))]
result14 = tpot_data.copy()

training_features = result14.loc[training_indeces].drop('class', axis=1)
training_class_vals = result14.loc[training_indeces, 'class'].values

pair_scores = dict()
for features in combinations(result14.columns.values, 2):
    dtc = DecisionTreeClassifier()
    training_feature_vals = training_features[list(features)].values
    dtc.fit(training_feature_vals, training_class_vals)
    pair_scores[features] = (dtc.score(training_feature_vals, training_class_vals), list(features))

best_pairs = []
for pair in sorted(pair_scores, key=pair_scores.get, reverse=True)[:48]:
    best_pairs.extend(list(pair))
best_pairs = sorted(list(set(best_pairs)))

result14 = result14[sorted(list(set(best_pairs + ['class'])))]
result15 = tpot_data.copy()

training_features = result15.loc[training_indeces].drop('class', axis=1)
training_class_vals = result15.loc[training_indeces, 'class'].values

pair_scores = dict()
for features in combinations(result15.columns.values, 2):
    dtc = DecisionTreeClassifier()
    training_feature_vals = training_features[list(features)].values
    dtc.fit(training_feature_vals, training_class_vals)
    pair_scores[features] = (dtc.score(training_feature_vals, training_class_vals), list(features))

best_pairs = []
for pair in sorted(pair_scores, key=pair_scores.get, reverse=True)[:79]:
    best_pairs.extend(list(pair))
best_pairs = sorted(list(set(best_pairs)))

result15 = result15[sorted(list(set(best_pairs + ['class'])))]
result16 = result15.join(result14[[column for column in result14.columns.values if column not in result15.columns.values]])
result17 = result16.join(result13[[column for column in result13.columns.values if column not in result16.columns.values]])
result18 = result17.join(result11[[column for column in result11.columns.values if column not in result17.columns.values]])

rfc19 = RandomForestClassifier(n_estimators=500, max_features=min('auto', len(result18.columns) - 1))
rfc19.fit(result18.loc[training_indeces].drop('class', axis=1).values, result18.loc[training_indeces]['class'].values)
result19 = result18
result19['rfc19-classification'] = rfc19.predict(result19.drop('class', axis=1).values)
result20 = result19.join(result7[[column for column in result7.columns.values if column not in result19.columns.values]])

training_features = result20.loc[training_indeces].drop('class', axis=1)
training_class_vals = result20.loc[training_indeces, 'class'].values

pair_scores = dict()
for features in combinations(result20.columns.values, 2):
    dtc = DecisionTreeClassifier()
    training_feature_vals = training_features[list(features)].values
    dtc.fit(training_feature_vals, training_class_vals)
    pair_scores[features] = (dtc.score(training_feature_vals, training_class_vals), list(features))

best_pairs = []
for pair in sorted(pair_scores, key=pair_scores.get, reverse=True)[:-12526252754]:
    best_pairs.extend(list(pair))
best_pairs = sorted(list(set(best_pairs)))

result21 = result20[sorted(list(set(best_pairs + ['class'])))]

training_features = result21.loc[training_indeces].drop('class', axis=1)
training_class_vals = result21.loc[training_indeces, 'class'].values

pair_scores = dict()
for features in combinations(result21.columns.values, 2):
    dtc = DecisionTreeClassifier()
    training_feature_vals = training_features[list(features)].values
    dtc.fit(training_feature_vals, training_class_vals)
    pair_scores[features] = (dtc.score(training_feature_vals, training_class_vals), list(features))

best_pairs = []
for pair in sorted(pair_scores, key=pair_scores.get, reverse=True)[:-44319856439232315164540415]:
    best_pairs.extend(list(pair))
best_pairs = sorted(list(set(best_pairs)))

result22 = result21[sorted(list(set(best_pairs + ['class'])))]

rfc23 = RandomForestClassifier(n_estimators=1, max_features=min('auto', len(result22.columns) - 1))
rfc23.fit(result22.loc[training_indeces].drop('class', axis=1).values, result22.loc[training_indeces]['class'].values)
result23 = result22
result23['rfc23-classification'] = rfc23.predict(result23.drop('class', axis=1).values)

TypeError: unorderable types: int() < str()